In [ ]:
import cv2
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

from scipy import ndimage
from PIL import Image
import glob
import imutils

for filename in glob.glob('/media/*.png'):
  frame=cv2.imread(filename)
  # Normalize the image
  img_normalized = cv2.normalize( frame, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F )
  cv2.imshow("orig",frame)
  cv2.imshow( "norm", img_normalized)
  cv2.waitKey(0
            )
# close all windows
cv2.destroyAllWindows()

In [ ]:
from Ultralytics import YOLO

model = YOLO('yolo8.pt')
results = model.train(data="/content/personDetection-1/data.yaml", epochs = 25,imgsz=224)

In [ ]:

cap = cv2.VideoCapture('/content/drive/MyDrive/KITTI-16-raw.mp4') #à modifier selon vos conditions

#read the video file until "q" is pressed
while( cap.isOpened() ):
    ret, frame = cap.read()
    #frame = cv2.resize(frame, (smt, smt), interpolation=cv.INTER_LINEAR) # add resize command to fit model size
    
    #Applying OpenCv thresholding https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html
    ret, thresh = cv2.threshold(frame, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    th2 = cv.adaptiveThreshold(frame,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,11,2)
    th3 = cv.adaptiveThreshold(frame,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,11,2)

    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('/content/drive/MyDrive/KITTI-16-raw.mp4') #à modifier selon vos conditions

#read the video file until "q" is pressed
while( cap.isOpened() ):
    ret, frame = cap.read()
    #frame = cv2.resize(frame, (smt, smt), interpolation=cv.INTER_LINEAR) # add resize command to fit model size
    results = model.predict(frame,conf=0.1)  # predict on an image

    if len(results) > 0:
        for r in results:
            res_plotted = results[ 0 ].plot() # get results
            box = r.boxes.cpu() #get bounding boxes
            box_sz = box.xyxy.numpy() # coordinates

            cv2.imshow(res_plotted)
            break


    c = cv2.waitKey(1)
    if c == 27:
        break

cap.release()
cv2.destroyAllWindows()
     